In [2]:
#uses RegisteredMultiplier  - v1.0x
#uses MultiplyAdd_NR - v1.1x

In [3]:
import sys
import math

In [4]:
#N>=2
N = input('Enter N: ')

Enter N: 10


In [5]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH default value (empty for 10): ')

Enter IN_WIDTH default value (empty for 10): 


In [6]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH default value (empty for 1): ')

Enter INPUT_REG_DEPTH default value (empty for 1): 


In [7]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH default value (empty for 1): ')

Enter MULT_PIPE_DEPTH default value (empty for 1): 


In [8]:
N = int(N)
if(N<2):
    N=2

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if IN_WIDTH<1:
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [9]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./DotProduct_Systolic_NR_{}.v".format(N), 'w+')

In [10]:
print("`timescale 1ns / 1ps\n", file=of)
print("module DotProduct_Systolic_NR_{}".format(N), file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output reg outReady = 0,", file=of)
lgn=math.ceil(math.log(N,2))
al=-1+lgn
if al==0:
    print("output reg signed [(2*IN_WIDTH):0] DP,", file=of)
else:
    print("output reg signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module DotProduct_Systolic_NR_10
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, 
input signed [IN_WIDTH-1:0] B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, 
output reg outReady = 0,
output reg signed [(2*IN_WIDTH)+3:0] DP,
output earlyOutReady
);



In [11]:
print("wire signed [(2*IN_WIDTH)-1:0] M0;", file=of)
print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH),", file=of)
print(".MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))", file=of)
print("Mult0(clk, reset, enable,", file=of)
print("inReady,", file=of)
print("A0, B0,", file=of)
print("MOR0, //not used", file=of)
print("M0,", file=of)
print("eMOR0); //not used", file=of)
print(file=of)

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))
Mult0(clk, reset, enable,
inReady,
A0, B0
MOR0, //not used
M0,
eMOR0); //not used



In [12]:
i=0; twoPW=1; j=N-1; k=0;
while(True):
    i+=1
    if i == twoPW:
        k+=1;
        if i==1:
            print("wire signed [2*IN_WIDTH:0] psl{} [0:{}];".format(k,twoPW-1), file=of)
        else:
            print("wire signed [(2*IN_WIDTH)+{}:0] psl{} [0:{}];".format(k-1,k,twoPW-1), file=of)
        twoPW = 2*twoPW
        j-=i
        if j==0:
            break
        elif j<twoPW:
            k+=1
            print("wire signed [(2*IN_WIDTH)+{}:0] psl{} [0:{}];".format(k-1,k,j-1), file=of)
            break
print(file=of)

wire signed [2*IN_WIDTH:0] psl1 [0:0];
wire signed [(2*IN_WIDTH)+1:0] psl2 [0:1];
wire signed [(2*IN_WIDTH)+2:0] psl3 [0:3];
wire signed [(2*IN_WIDTH)+3:0] psl4 [0:1];



In [13]:
i=0; twoPW=1; k=0; ok=0; l=0; ol=0;
while(i<N-1):
    i+=1
    ol=l;
    l+=1;
    ok=k;
    if i == twoPW:
        k+=1;
        twoPW = 2*twoPW
        l=0;
    if ok==0:
        print("MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH(2*IN_WIDTH), ", end='', file=of)
    else:
        print("MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH((2*IN_WIDTH)+{}), ".format(ok), end='', file=of)
    print(".OUT_WIDTH((2*IN_WIDTH)+{}), ".format(k), end='', file=of)
    print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )", file=of)
    print("MA{}(clk, reset, enable,".format(i), file=of)
    print("inReady,", file=of)
    if i ==1:
        print("A1, B1,\nM0,".format(i))
    else:
        print("A{}, B{},\npsl{}[{}],".format(i, i, ok,ol, i))
    if i==N-1:
        print("earlyOutReady,", file=of)
    else:
        print("OR{}, //not used".format(i))
    print("psl{}[{}],".format(k,l), file=of)
    print("eOR{}); //not used".format(i))
    print(file=of)

MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH(2*IN_WIDTH), .OUT_WIDTH((2*IN_WIDTH)+1), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
MA1(clk, reset, enable,
inReady,
A1, B1,
M0,
OR1, //not used
psl1[0],
eOR1); //not used

MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH((2*IN_WIDTH)+1), .OUT_WIDTH((2*IN_WIDTH)+2), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
MA2(clk, reset, enable,
inReady,
A2, B2,
psl1[0],
OR2, //not used
psl2[0],
eOR2); //not used

MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH((2*IN_WIDTH)+2), .OUT_WIDTH((2*IN_WIDTH)+2), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
MA3(clk, reset, enable,
inReady,
A3, B3,
psl2[0],
OR3, //not used
psl2[1],
eOR3); //not used

MultiplyAdd_NR #( .IN_M_WIDTH(IN_WIDTH), .IN_A_WIDTH((2*IN_WIDTH)+2), .OUT_WIDTH((2*IN_WIDTH)+3), .INPUT_REG_DEPTH(INPUT_REG_DEPTH), .MULT_PIPE_DEPTH(MULT_PIPE_DEPTH) )
MA4(clk, reset, enable,
inReady,
A4, B4,
psl2[1],
O

In [14]:
print("always @(posedge clk) begin", file=of)
print("\tif(reset) begin", file=of)
print("\t\toutReady <= 0;", file=of)
print("\tend", file=of)
print("\telse if(enable) begin", file=of)
print("\t\toutReady <= earlyOutReady;", file=of)
print("\t\tif(earlyOutReady) begin", file=of)
print("\t\t\tDP <= psl{}[{}];".format(k,l), file=of)
print("\t\tend", file=of)
print("\tend", file=of)
print("end\n", file=of)

always @(posedge clk) begin
	if(reset) begin
		outReady <= 0;
	end
	else if(enable) begin
		outReady <= earlyOutReady;
		if(earlyOutReady) begin
			DP <= psl4[1];
		end
	end
end



In [ ]:
print("endmodule", file=of)

In [15]:
if __Print_To_File>0:
    of.close()